In [3]:
import pdfplumber
import re
import json

In [4]:
folder = r'C:\path\to\folder'
mypdf = folder + '\\' + 'filename.pdf'

In [ ]:
pack_slip = {}
ps_items = []
with pdfplumber.open(mypdf) as pdf:
    for a_page in pdf.pages:
        if a_page.page_number == 1:
            line1_ship_to = None
            line2_ship_to = None
            line3_ship_to = None
            line4_ship_to = None
            line5_ship_to = None
            collect = False
            for line in a_page.extract_text_simple().split('\n'):
                if 'BILL TO' in line:
                    collect = False
                    break
                elif 'SHIP TO' in line:
                    line1_ship_to = line.split('SHIP TO:')[1].strip()
                    collect = True
                    pass
                elif collect == True:
                    if 'SHIP FROM' in line:
                        line2_ship_to = line.split('SHIP FROM')[0].strip()
                    elif 'street address' in line:
                        line3_ship_to = line.split('2905')[0].strip()
                    elif 'city address' in line:
                        line4_ship_to = line.split('Beaverton')[0].strip()
                    else:
                        line5_ship_to = line.strip()
            full_ship_to = str(line1_ship_to) + ', ' + str(line2_ship_to) + ', ' + str(line3_ship_to) + ', ' + str(line4_ship_to) + ', ' + str(line5_ship_to)
            full_ship_to = full_ship_to.replace('None, ',' ')
        for table in a_page.extract_tables():
            for row in table:
                if row[0].startswith('Delivery Name'):
                    delivery_id = re.split(' |\n',row[0])[2]
                    for elem in row:
                        if isinstance(elem,str) and elem.startswith('Date Printed'):
                            print_date = row[2][13:]
                elif row[0] == 'SHIP FROM:':
                    full_ship_from = row[1]
                    line1_ship_from = full_ship_from.split('\n')[0]
                    line2_ship_from = full_ship_from.split('\n')[1]
                    line3_ship_from = full_ship_from.split('\n')[2]
                elif row[0] == 'BILL TO:':
                    full_bill_to = row[1]
                    line1_bill_to = full_bill_to.split('\n')[0]
                    line2_bill_to = full_bill_to.split('\n')[1]
                    line3_bill_to = full_bill_to.split('\n')[2]
                elif row[0].startswith('PO Ref #'):
                    purchase_order_number = row[0].split('\n')[-1]
                    ship_method = row[1].split('\n')[-1]
                    freight_terms = row[2].split('\n')[-1]
                    sales_order = row[3].split('\n')[-1]
                elif 'Shipping Instruction' in row[0]:
                    packing_slip_number = row[1].split('\n')[-1]
                    for elem in row:
                        if isinstance(elem,str) and elem.startswith('Tracking'):
                            tracking_numbers = ''.join(elem.split('\n')).replace('Tracking Number','').split(':')
                elif row[0] not in ['Packing Slip','Customer Item #/Desc','Line Description','']:
                    ps_items.append({'part_number': row[0]
                                     ,'purchase_order_line_number': row[1]
                                     ,'tek_cso_item_number': row[2]
                                     ,'lot_number': ''.join(row[4].split('\n'))
                                     ,'quantity_ordered': row[7]
                                     ,'quantity_shipped': row[8]
                                    })
pack_slip = {
    "delivery_id": delivery_id
    ,"print_date": print_date
    ,"full_ship_to": full_ship_to
    ,"line1_ship_to": line1_ship_to
    ,"line2_ship_to": line2_ship_to
    ,"line3_ship_to": line3_ship_to
    ,"full_ship_from": full_ship_from.replace('\n',', ')
    ,"line1_ship_from": line1_ship_from
    ,"line2_ship_from": line2_ship_from
    ,"line3_ship_from": line3_ship_from
    ,"full_bill_to": full_bill_to.replace('\n',', ')
    ,"line1_bill_to": line1_bill_to
    ,"line2_bill_to": line2_bill_to
    ,"line3_bill_to": line3_bill_to
    ,"purchase_order_number": purchase_order_number
    ,"ship_method": ship_method
    ,"freight_terms": freight_terms
    ,"sales_order": sales_order
    ,"packing_slip_number": packing_slip_number
    ,"tracking_numbers": tracking_numbers
}

In [ ]:
pack_slip_json = json.dumps(pack_slip)
ps_items_json = json.dumps(ps_items)